In [1]:
import re
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/steve/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/steve/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
f_shakespeare = open('../../data/shakespeare_cleaned.txt','r')
shakespeare = f_shakespeare.read()

f_obama = open('../../data/obama_cleaned.txt','r')
obama = f_obama.read()



In [70]:
n = 70 #amount of words in each data point
obama_words = obama.split(' ')
c_obama = [" ".join(obama_words[i:i+n]) for i in range(0, len(obama_words), n)]

print(np.average([len(x) for x in c_obama]))
print(len(c_obama))


384.4099099099099
444


In [71]:
n = 68 #amount of words/sentences in each data point
shake_words = shakespeare.split(' ')

#split based on n sentences - disabled for now
#c_shakespeare = [". ".join(shake_words[i:i+n]) for i in range(0, len(shake_words), n)]

#split based on n words
c_shakespeare = [" ".join(shake_words[i:i+n]) for i in range(0, len(shake_words), n)]

np.random.seed(59374)
np.random.shuffle(c_shakespeare)

print(np.average([len(x) for x in c_shakespeare]))
print(len(c_shakespeare))


370.10420193935664
12994


In [5]:
#naive bayes - big failure but good practice

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopset = list(set(stopwords.words('english')))

def word_feats(words):
    return dict([(word, True) for word in words.split() if word.lower() not in stopset])

l_shakespeare = [(word_feats(x), "shakespeare") for x in c_shakespeare]
l_obama = [(word_feats(x), "obama") for x in c_obama]

total_data = l_shakespeare[:444] + l_obama
np.random.seed(59374)
np.random.shuffle(total_data)

train_set, test_set = train_test_split(total_data)

classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)


1.0
Most Informative Features
                   shall = True           shakes : obama  =     35.4 : 1.0
                  people = True            obama : shakes =     27.5 : 1.0
                    also = True            obama : shakes =     22.3 : 1.0
                children = True            obama : shakes =     17.8 : 1.0
                    want = True            obama : shakes =     14.6 : 1.0


In [104]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
import operator
import random

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles) if t not in string.punctuation and not any(char.isdigit() for char in t)]

def generate_vocab(data, vocab_size=None):
    """
    Generate vocabulary from a given string
    :param data: List of strings (should be denoised)
    :param vocab_size: Number of entries to save
    :return: Dictionary with words as the key and index as output
    """
    word_count = {}  # count of each word encountered
    word_index = {}  # vocabulary

    for word in data.split():
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
    if vocab_size is not None:
        ordered_words_by_index = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)
        ordered_words_by_index = ordered_words_by_index[:vocab_size]
        for word in ordered_words_by_index:
            word_index[word[0]] = len(word_index)
    else:
        for word in word_count:
            word_index[word] = len(word_index)
    return word_index

#total_data = [(x, "shakespeare") for x in c_shakespeare[:444]] + [(x, "obama") for x in c_obama]
q_shakespeare = [(x, "shakespeare") for x in c_shakespeare]
q_obama = [(x, "obama") for x in c_obama]
new_total = q_shakespeare[:444] + q_obama
np.random.seed(59374)
np.random.shuffle(new_total)

total_dict = {"text" : [x[0] for x in new_total], "author" : [x[1] for x in new_total]}

#corpus = ""
#for block in total_data:
#    corpus = corpus + block

#word_sequence = text_to_word_sequence(corpus)
#words = set(total_words)
#vocab_size = len(words)
#tokenizer = Tokenizer(num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')

#final = one_hot(corpus, round(vocab_size*1.4))
#final = to_categorical(final)
#print(len(words))

#vocab = generate_vocab(corpus, vocab_size)
np.random.seed(59374)
np.random.shuffle(new_total)

#vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())                         stop_words='english')
#X = vectorizer.fit_transform(total_data)
#print(X[:50])
#train_set, test_set = train_test_split(total_data)

#X = tokenizer.texts_to_sequences()
#X = pad_sequences(X)



In [124]:
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.preprocessing import text
from sklearn.preprocessing import LabelEncoder
from keras import utils

def tokenize_input(train, test, vocab):
    # tokenize input text
    tokenizer = text.Tokenizer(num_words=vocab)
    tokenizer.fit_on_texts(train)
    # create input matrix
    return tokenizer.texts_to_matrix(train), \
           tokenizer.texts_to_matrix(test), \
           tokenizer

def encode_labels(train, test):
    encoder = LabelEncoder()
    encoder.fit(train)
    y_train = encoder.transform(train)
    num_classes = np.max(y_train) + 1
    return utils.to_categorical(y_train, num_classes), \
           utils.to_categorical(encoder.transform(test), num_classes), \
           encoder.classes_, num_classes

def split_dataset(dataset, split, input_field, label_field, ):
    train_size = int(len(dataset[input_field]) * split)
    return dataset[input_field][:train_size], dataset[input_field][train_size:], \
           dataset[label_field][:train_size], dataset[label_field][train_size:]

def init_data(data, vocab_size, data_split):
    train_input, test_input, train_label, test_label = split_dataset(total_dict, data_split, 'text', 'author')
    x_train, x_test, tokenizer = tokenize_input(train_input, test_input, vocab_size)
    y_train, y_test, text_labels, num_classes = encode_labels(train_label, test_label)
    return train_input, test_input, train_label, test_label, x_train, x_test, y_train, y_test, tokenizer, text_labels,\
           num_classes

train_input, test_input, train_label, test_label, x_train, x_test, y_train, y_test, tokenizer, text_labels, \
    num_classes = init_data(total_dict, 8603, .7)

model = Sequential([
    Dense(512, input_shape=(8603,)),
    Activation('relu'), #TODO: change to sigmoid?
    Dense(2),
    Activation('softmax')
])
model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['categorical_accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=256, verbose=1)

scores = model.evaluate(x_test, y_test, verbose=1)
#pred = model.predict(x_test)
#print(pred)
#print("Accuracy: %.2f%%" % (scores[1] * 100))
#model.fit(x_train,y_train,epochs=2000,batch_size=5,validation_split=0.05,verbose=0);
#print(model.summary())

Epoch 1/20
621/621 [==============================] - 2s 3ms/step - loss: 0.6022 - categorical_accuracy: 0.7375
Epoch 2/20
621/621 [==============================] - 0s 765us/step - loss: 0.2166 - categorical_accuracy: 0.9984
Epoch 3/20
621/621 [==============================] - 0s 784us/step - loss: 0.0593 - categorical_accuracy: 1.0000
Epoch 4/20
621/621 [==============================] - 0s 778us/step - loss: 0.0154 - categorical_accuracy: 1.0000
Epoch 5/20
621/621 [==============================] - 0s 803us/step - loss: 0.0048 - categorical_accuracy: 1.0000
Epoch 6/20
621/621 [==============================] - 0s 761us/step - loss: 0.0019 - categorical_accuracy: 1.0000
Epoch 7/20
621/621 [==============================] - 1s 1ms/step - loss: 9.3159e-04 - categorical_accuracy: 1.0000
Epoch 8/20
621/621 [==============================] - 1s 1ms/step - loss: 5.2699e-04 - categorical_accuracy: 1.0000
Epoch 9/20
621/621 [==============================] - 1s 1ms/step - loss: 3.3101e-04 -